In [1]:
from accelerate import Accelerator

import sys 
sys.path.append('../scripts')
from train import load_dataset

In [2]:
# load ujindoor datasets
dataset_name_0_0 = "ujindoorloc_building_0_floor_0"
train_loader, test_loader = load_dataset(name=dataset_name_0_0, k=5, batch_size = 32, train_ratio=0.8, test_ratio=0.2)


Processing...
100%|██████████| 7773/7773 [00:07<00:00, 1074.50it/s]
Done!


In [3]:
from tqdm import tqdm
import torch
import torch.optim as optim
import numpy as np

In [4]:
for input in train_loader:
    y = input.y
    print(y.shape)
    y = y.reshape(-1, 2)
    # flag = y.shape[0] if y.shape[0] != 32 else None
    # if flag:
    #     print(flag)

torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32, 2])


In [4]:
def train(model, name, train_loader, val_loader, epochs, lr=0.001, mae_loss=True):

    accelerator = Accelerator()
    # set optimizer and loss function 
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # loss functions
    mae = torch.nn.L1Loss()
    mse = torch.nn.MSELoss()

    # accelerator 
    model, optimizer, train_loader, val_loader = accelerator.prepare(model, optimizer, train_loader, val_loader)


    train_loss_epoch_mae = []
    train_loss_epoch_mse = []
    val_loss_epoch_mae = []
    val_loss_epoch_mse = []
    for epoch in range(epochs):

        # set model to train
        model.train()

        # collect losses
        train_losses_mae = []
        train_losses_mse = []

        # set tqdm loop 
        loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)

        for batch_idx, data in loop:

            # get data
            x, edge_attr, edge_index, batch_index, y = data.x, data.edge_attr, data.edge_index, data.batch, data.y

            # reshape y
            y = y.reshape(-1, 2)

            # get output
            output = model(x, edge_attr, edge_index, batch_index)

            # calculate loss for x and y  dims
            loss_mae = mae(output[:, :2], y[:, :2])
            loss_mse = mse(output[:, :2], y[:, :2])

            # collect losses
            train_losses_mae.append(accelerator.gather(loss_mae.item()))
            train_losses_mse.append(accelerator.gather(loss_mse.item()))

            # zero grad
            optimizer.zero_grad()

            # backpropagate
            if mae_loss:
                accelerator.backward(loss_mae)
            else:
                accelerator.backward(loss_mse)

            # optimizer step
            optimizer.step()

            # set description for tqdm
            loop.set_description(f"Epoch [{epoch + 1}/{epochs}]")
            loop.set_postfix(loss_mae=loss_mae.item(), loss_mse=loss_mse.item())
        
        # get avg losses
        avg_train_mae = np.mean(train_losses_mae)
        avg_train_mse = np.mean(train_losses_mse)

        # append to epoch losses
        train_loss_epoch_mae.append(avg_train_mae)
        train_loss_epoch_mse.append(avg_train_mse)

        # evaluate on validation set
        model.eval()

        # collect losses
        val_losses_mae = []
        val_losses_mse = []

        # set tqdm loop
        val_loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=False)
        
        # torch no grad
        with torch.no_grad():
            for batch_idx, data in val_loop:

                # get data
                x, edge_attr, edge_index, batch_index, y = data.x, data.edge_attr, data.edge_index, data.batch, data.y

                # reshape y
                y = y.reshape(-1, 2)

                # get output
                output = model(x, edge_attr, edge_index, batch_index)

                # calculate loss for x and y dims
                loss_mae = mae(output[:, :2], y[:, :2])
                loss_mse = mse(output[:, :2], y[:, :2])

                # collect losses
                val_losses_mae.append(accelerator.gather(loss_mae.item()))
                val_losses_mse.append(accelerator.gather(loss_mse.item()))

                # set description for tqdm
                val_loop.set_description(f"Epoch [{epoch + 1}/{epochs}]")
                val_loop.set_postfix(loss_mae=loss_mae.item(), loss_mse=loss_mse.item())

        # get avg losses
        avg_val_mae = np.mean(val_losses_mae)
        avg_val_mse = np.mean(val_losses_mse)

        # append to epoch losses
        val_loss_epoch_mae.append(avg_val_mae)
        val_loss_epoch_mse.append(avg_val_mse)

    # wait for accelerator to finish
    accelerator.wait_for_everyone()

    # save model if main process
    if accelerator.is_main_process:
        torch.save(model.state_dict(), f"./weights/{name}.pt")

    # return losses
    return train_loss_epoch_mae, train_loss_epoch_mse, val_loss_epoch_mae, val_loss_epoch_mse



        


In [5]:
from accelerate import notebook_launcher

In [6]:
from model import GCN
model = GCN(3, 32, 128, 2, 2, 0.3)

name = "Building_0_floor_0_model_mae"
epochs = 500
lr = 0.001
mae_loss = True
args = (model, name, train_loader, test_loader, epochs, lr, mae_loss)
notebook_launcher(train, args, num_processes=1)


2
Launching training on one GPU.


KeyboardInterrupt: 